In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [6]:
URL = "https://www.indeed.com/jobs?q=software+engineer"
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, "html.parser")
#printing soup in a more structured tree format that makes for easier reading
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type">
   <script src="/s/d8ca20f/en_US.js" type="text/javascript">
   </script>
   <link href="/s/ecdfb5e/jobsearch_all.css" rel="stylesheet" type="text/css">
    <link href="http://rss.indeed.com/rss?q=software+engineer" rel="alternate" title="Software Engineer Jobs, Employment" type="application/rss+xml">
     <link href="/m/jobs?q=software+engineer" media="only screen and (max-width: 640px)" rel="alternate">
      <link href="/m/jobs?q=software+engineer" media="handheld" rel="alternate">
       <script type="text/javascript">
        if (typeof window['closureReadyCallbacks'] == 'undefined') {
        window['closureReadyCallbacks'] = [];
    }

    function call_when_jsall_loaded(cb) {
        if (window['closureReady']) {
            cb();
        } else {
            window['closureReadyCallbacks'].push(cb);
        }
    }
       </script>
       <meta content="1" name="ppstri

In [10]:
def extract_job_title_from_result(soup): 
  jobs = []
  for div in soup.find_all(name="div", attrs={"class":"row"}):
      for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
        jobs.append(a["title"])
  return(jobs)

extract_job_title_from_result(soup)

[u'Software Engineer',
 u'Software Engineer - Full Stack Web Developer',
 u'Software Engineer (full stack node.js)',
 u'Python Developer - Software Engineer',
 u'Engineer, Software Development',
 u'Java Systems Engineer',
 u'New Graduate \u2013 Product Development (Computer Science)',
 u'Software Engineer',
 u'Software Engineer',
 u'Jr. Software Engineer',
 u'Software Engineer (University)',
 u'Java Developer',
 u'Software Engineer, Web',
 u'Software Engineer',
 u'Mathematical/Statistical Software Developer',
 u'Backend Developer / Server Expert (Techstars Company)']

In [12]:
def extract_company_from_result(soup):
  companies = []
  for div in soup.find_all(name="div", attrs={"class":"row"}):
    company = div.find_all(name="span", attrs={"class":"company"})
    if len(company) > 0:
      for b in company:
        companies.append(b.text.strip())
    else:
      sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
      for span in sec_try:
            companies.append(span.text.strip())
            
  return(companies)
 
extract_company_from_result(soup)

[u'Indeed Prime',
 u'IntelliFarms',
 u'Pluralsight',
 u'Exeter Government Services LLC',
 u'TiVo',
 u'Wells Fargo',
 u'Egnyte, Inc.',
 u'Cisco',
 u'Tesla',
 u'Keyo',
 u'Adobe',
 u'Aroha Technologies Inc',
 u'Marqeta',
 u'Microsoft',
 u'Hygiena',
 u'Hidrate Inc.']

In [14]:
def extract_location_from_result(soup): 
  locations = []
  spans = soup.findAll("span", attrs={"class": "location"})
  for span in spans:
    locations.append(span.text)
  return(locations)

extract_location_from_result(soup)

[u'San Francisco, CA',
 u'Olathe, KS',
 u'South Jordan, UT',
 u'Alexandria, VA',
 u'San Jose, CA 95131 (North Valley area)',
 u'Fremont, CA 94537 (Cabrillo area)',
 u'Mountain View, CA 94043',
 u'Milpitas, CA',
 u'Fremont, CA',
 u'Palo Alto, CA',
 u'San Jose, CA',
 u'Mountain View, CA',
 u'Oakland, CA',
 u'Palo Alto, CA',
 u'Wilmington, DE',
 u'Minneapolis, MN']

In [15]:
def extract_salary_from_result(soup): 
  salaries = []
  for div in soup.find_all(name="div", attrs={"class":"row"}):
    try:
      salaries.append(div.find("nobr").text)
    except:
      try:
        div_two = div.find(name="div", attrs={"class":"sjcl"})
        div_three = div_two.find("div")
        salaries.append(div_three.text.strip())
      except:
        salaries.append("Nothing_found")
  return(salaries)
extract_salary_from_result(soup)

['Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found']

In [18]:
def extract_summary_from_result(soup): 
  summaries = []
  spans = soup.findAll("span", attrs={"class": "summary"})
  for span in spans:
    summaries.append(span.text.strip())
  return(summaries)
extract_summary_from_result(soup)

[u'How Indeed Prime Works Apply to Prime in 5 minutes. Apply to 100+ top companies with 1 simple application to Indeed Prime....',
 u'Be part of a team that cares about who you are and your growth as a software engineer. Receive great benefits (401k match, profit sharing, catered lunches, beer...',
 u"Continuous Delivery - teams independently ship code to prod every day. We do this through the tech industry's leading learning platform for serious Developer,...",
 u'Exeter is seeking a Python Developer, Software Engineer to support our Alexandria, VA office. Software Development Plan, Software Integration Plan, Software...',
 u'System Software Engineer. As a System Software Engineer in platform technolgies software team, you will be developing systems software to enable cutting edge...',
 u'Develop, maintain and support COTLS Java application. Wells Fargo Wholesale Loan Services Technology division is seeking a Java/J2EE engineer to provide...',
 u'Knowledge of Java and/or Python. Pleas

In [26]:
max_results_per_city = 100
city_set = ["New+York","Chicago","San+Francisco", "Austin"]
columns = ["city", "job_title", "company_name", "location", "summary", "salary"]
sample_df = pd.DataFrame(columns = columns)

In [28]:
#scraping code:
for city in city_set:
  for start in range(0, max_results_per_city, 10):
      page = requests.get("http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=" + str(city) + "&start=" + str(start))
      time.sleep(1)  #ensuring at least 1 second between page grabs
      soup = BeautifulSoup(page.text, "lxml", from_encoding="utf-8")
      for div in soup.find_all(name="div", attrs={"class":"row"}): 
        #specifying row num for index of job posting in dataframe
        num = (len(sample_df) + 1) 
        #creating an empty list to hold the data for each posting
        job_post = [] 
        #append city name
        job_post.append(city) 
        #grabbing job title
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"]) 
        #grabbing company name
        company = div.find_all(name="span", attrs={"class":"company"}) 
        if len(company) > 0: 
            for b in company:
                job_post.append(b.text.strip()) 
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                job_post.append(span.text) 
        #grabbing location name
        c = div.findAll("span", attrs={"class": "location"}) 
        for span in c: 
            job_post.append(span.text) 
        #grabbing summary text
        d = div.findAll("span", attrs={"class": "summary"}) 
        for span in d:
            job_post.append(span.text.strip()) 
        #grabbing salary
        try:
            job_post.append(div.find("nobr").text) 
        except:
            try:
                div_two = div.find(name="div", attrs={"class":"sjcl"}) 
                div_three = div_two.find("div") 
                job_post.append(div_three.text.strip())
            except:
                job_post.append("Nothing_found") 
        #appending list of job post info to dataframe at index num
        sample_df.loc[num] = job_post

#saving sample_df as a local csv file — define your own local path to save contents 
sample_df.to_csv("scraped-jobs.csv", encoding="utf-8")

In [29]:
def extract_job(jobId):
    URL = "https://www.indeed.com/jobs?q=sw&vjk=" + str(jobId)
    #conducting a request of the stated URL above:
    page = requests.get(URL)
    #specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
    soup = BeautifulSoup(page.text, "html.parser")
    #printing soup in a more structured tree format that makes for easier reading
    print(soup.prettify())

extract_job("febf687db2ce1f38")

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type">
   <script src="/s/d8ca20f/en_US.js" type="text/javascript">
   </script>
   <link href="/s/ecdfb5e/jobsearch_all.css" rel="stylesheet" type="text/css">
    <link href="http://rss.indeed.com/rss?q=sw" rel="alternate" title="Sw Jobs, Employment" type="application/rss+xml">
     <link href="/m/jobs?q=sw" media="only screen and (max-width: 640px)" rel="alternate">
      <link href="/m/jobs?q=sw" media="handheld" rel="alternate">
       <script type="text/javascript">
        if (typeof window['closureReadyCallbacks'] == 'undefined') {
        window['closureReadyCallbacks'] = [];
    }

    function call_when_jsall_loaded(cb) {
        if (window['closureReady']) {
            cb();
        } else {
            window['closureReadyCallbacks'].push(cb);
        }
    }
       </script>
       <meta content="1" name="ppstriptst">
        <script src="/s/d53fa03/jobsearch-all-compile